In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import ConvexHull
import pandas as pd
import nibabel as nib
from glob import glob
from tqdm import tqdm

In [ ]:
files = glob("Data/Pre-processed/**/*.nii.gz")
files = [f for f in files if "CN-FSL" not in f]
images = []
labels = []
# Find the brain volume (concave and convex)
def brain_volume(im, voxel_spacing=(1.0, 1.0, 1.0)):
    # Get coordinates of non-zero voxels
    coords = np.column_stack(np.where(im>0.02))
    hull = ConvexHull(coords*voxel_spacing)
    return {"concave":len(coords)*np.prod(voxel_spacing), "convex":hull.volume}

vols = []

for fp in tqdm(files):
    # images.append()
    img = nib.load(fp,mmap=False)
    data = img.get_fdata()
    spacing = img.header.get_zooms()
    group = 0
    if "MCI\\" in fp: group = 1
    if "AD\\" in fp: group = 2
    # labels.append(group)
    
    vols.append(brain_volume(data, spacing))

100%|██████████| 2000/2000 [20:26<00:00,  1.63it/s]


In [7]:

volumes = pd.DataFrame(vols)
volumes

,concave,convex
0,1631980.0,1.770903e+06
1,1628163.0,1.767325e+06
2,1580466.0,1.704634e+06
3,1477454.0,1.629739e+06
4,1595403.0,1.739119e+06
...,...,...
1995,1654017.0,1.797397e+06
1996,1597718.0,1.738169e+06
1997,1601268.0,1.727674e+06
1998,1609039.0,1.749592e+06


In [8]:
volumes["subject"] = pd.read_csv("Data/subjects.csv")["subject"]
volumes.to_csv("Data/Individual level/brain_volume.csv")
volumes

,concave,convex,subject
0,1631980.0,1.770903e+06,S6546
1,1628163.0,1.767325e+06,S6570
2,1580466.0,1.704634e+06,S6570
3,1477454.0,1.629739e+06,S6574
4,1595403.0,1.739119e+06,S6574
...,...,...,...
1995,1654017.0,1.797397e+06,S0300
1996,1597718.0,1.738169e+06,S0916
1997,1601268.0,1.727674e+06,S1262
1998,1609039.0,1.749592e+06,S0404
